# 进程、线程

In [ ]:
# 行程与线程

In [ ]:
# 对于操作系统来说，一个任务就是一个进程（Process），比如打开一个浏览器就是启动一个浏览器进程。
# 有些进程还不止同时干一件事，比如Word，它可以同时进行打字、拼写检查、打印等事情。
# 在一个进程内部，要同时干多件事，就需要同时运行多个“子任务”，我们把进程内的这些“子任务”称为线程（Thread）。

In [9]:
# 进程与线程的不同：
# 简而言之,一个程序至少有一个进程,一个进程至少有一个线程.
# ​ 进程就是一个应用程序在处理机上的一次执行过程，它是一个动态的概念，而线程是进程中的一部分，进程包含多个线程在运行。
# ​ 多线程可以共享全局变量，多进程不能。多线程中，所有子线程的进程号相同；多进程中，不同的子进程进程号不同

# 多进程并发、多线程并发

In [10]:
# 多进程并发
# 使用多进程并发是将一个应用程序划分为多个独立的进程（每个进程只有一个线程），这些独立的进程间可以互相通信，共同完成任务。
# 由于操作系统对进程提供了大量的保护机制，以避免一个进程修改了另一个进程的数据，使用多进程比多线程更容易写出安全的代码。
# 但这也造就了多进程并发的两个缺点：
# 1.在进程件的通信，无论是使用信号、套接字，还是文件、管道等方式，其使用要么比较复杂，要么就是速度较慢或者两者兼而有之。
# 2.运行多个线程的开销很大，操作系统要分配很多的资源来对这些进程进行管理。
# 由于多个进程并发完成同一个任务时，不可避免的需要操作同一个数据在进程间相互通信，上述的两个缺点也就决定了多进程并发不是一个好的选择。

# 多线程并发
# 多线程并发指的是在同一个进程中执行多个线程。有操作系统相关知识的应该知道，线程是轻量级的进程，
# 每个线程可以独立的运行不同的指令序列，但是线程不独立的拥有资源，依赖于创建它的进程而存在。
# 也就是说，同一进程中的多个线程共享相同的地址空间，可以访问进程中的大部分数据，指针和引用可以在线程间进行传递。
# 这样，同一进程内的多个线程能够很方便的进行数据共享以及通信，也就比进程更适用于并发操作。
# 由于缺少操作系统提供的保护机制，在多线程共享数据及通信时，就需要程序员做更多的工作以保证对共享数据段的操作是以预想的操作顺序进行的，
# 并且要极力的避免死锁(deadlock)。

# Python多线程

In [ ]:
# 在Python多线程下，每个线程的执行方式：
# 1.获取GIL
# 2.执行代码直到sleep或者是python虚拟机将其挂起。
# 3.释放GIL
# 可见，某个线程想要执行，必须先拿到GIL，我们可以把GIL看作是“通行证”，并且在一个python进程中，GIL只有一个。
# 拿不到通行证的线程，就不允许进入CPU执行。

In [ ]:
# CPU密集型代码(各种循环处理、计数等等)，在这种情况下，由于计算工作多，ticks计数很快就会达到阈值，
# 然后触发GIL的释放与再竞争（多个线程来回切换当然是需要消耗资源的），所以python下的多线程对CPU密集型代码并不友好。

# IO密集型代码(文件处理、网络爬虫等)，多线程能够有效提升效率(单线程下有IO操作会进行IO等待，造成不必要的时间浪费，
# 而开启多线程能在线程A等待时，自动切换到线程B，可以不浪费CPU的资源，从而能提升程序执行效率)。所以python的多线程对IO密集型代码比较友好。

# 而在python3.x中，GIL不使用ticks计数，改为使用计时器（执行时间达到阈值后，当前线程释放GIL），
# 这样对CPU密集型程序更加友好，但依然没有解决GIL导致的同一时间只能执行一个线程的问题，所以效率依然不尽如人意。

In [ ]:
# 请注意：多核多线程比单核多线程更差，原因是单核下的多线程，每次释放GIL，唤醒的那个线程都能获取到GIL锁，所以能够无缝执行，
# 但多核下，CPU0释放GIL后，其他CPU上的线程都会进行竞争，但GIL可能会马上又被CPU0拿到，
# 导致其他几个CPU上被唤醒后的线程会醒着等待到切换时间后又进入待调度状态，这样会造成线程颠簸(thrashing)，导致效率更低。

# 所以说：“python下想要充分利用多核CPU，就用多进程”，原因是什么呢？
# 原因是：每个进程有各自独立的GIL，互不干扰，这样就可以真正意义上的并行执行，
# 所以在python中，多进程的执行效率优于多线程(仅仅针对多核CPU而言)。

# 所以在这里说结论：多核下，想做并行提升效率，比较通用的方法是使用多进程，能够有效提高执行

In [ ]:
# 尽管Python完全支持多线程编程， 但是解释器的C语言实现部分在完全并行执行时并不是线程安全的，所以这时候才引入了GIL
# 解释器被一个全局解释器锁保护着，它确保任何时候都只有一个Python线程执行(保证C实现部分能线程安全)
# GIL最大的问题就是Python的多线程程序并不能利用多核CPU的优势 （比如一个使用了多个线程的计算密集型程序只会在一个单CPU上面运行）
# 注意：GIL只会影响到那些严重依赖CPU的程序（比如计算型的）如果你的程序大部分只会涉及到I/O，比如网络交互，
# 那么使用多线程就很合适 ~ 因为它们大部分时间都在等待（线程被限制到同一时刻只允许一个线程执行这样一个执行模型。
# GIL会根据执行的字节码行数和时间片来释放GIL，在遇到IO操作的时候会主动释放权限给其他线程）
# 所以Python的线程更适用于处理I/O和其他需要并发执行的阻塞操作，而不是需要多处理器并行的计算密集型任务
# （对于IO操作来说，多进程和多线程性能差别不大）【计算密集现在可以用Python的Ray框架】

# 并发、并行

In [ ]:
# 并发与并行
# 当有多个线程在操作时，如果系统只有一个 CPU，则它根本不可能真正同时进行一个以上的线程，
# 它只能把 CPU 运行时间划分成若干个时间段，再将时间段分配给各个线程执行，在一个时间段的线程代码运行时,
# 其它线程处于挂起状态.这种方式我们称之为并发（Concurrent）。
# 当系统有一个以上 CPU 时，则线程的操作有可能非并发。当一个 CPU 执行一个线程时，另一个 CPU 可以执行另一个线程，
# 两个线程互不抢占 CPU 资源，可以同时进行，这种方式我们称之为并行（Parallel）。

# ！！！解释一：并行是指两个或者多个事件在同一时刻发生；而并发是指两个或多个事件在同一时间间隔发生。
# 解释二：并行是在不同实体上的多个事件，并发是在同一实体上的多个事件。
# 解释三：并发是在一台处理器上“同时”处理多个任务，并行是在多台处理器上同时处理多个任务。如 hadoop 分布式集群。
# 所以并发编程的目标是充分的利用处理器的每一个核，以达到最高的处理性能

# 在单核时代，多个线程是并发的，在一个时间段内轮流执行；在多核时代，多个线程可以实现真正的并行，在多核上真正独立的并行执行。
# 例如现在常见的4核4线程可以并行4个线程；4核8线程则使用了超线程技术，把一个物理核模拟为2个逻辑核心，可以并行8个线程。

# 计算密集型、IO密集型

In [ ]:
# 计算密集型，顾名思义就是应用需要非常多的CPU计算资源，在计算密集型任务的特点是要进行大量的计算，消耗CPU资源，
# 比如计算圆周率、对视频进行高清解码等等，全靠CPU的运算能力。

# 对于IO密集型的应用，涉及到网络、磁盘IO的任务都是IO密集型任务，大多消耗都是硬盘读写和网络传输的消耗。

# 那么GIL多线程的不足，其实是对于计算密集型的不足，这个解决可以利用多进程进行解决，
# 而对于IO密集型的任务，我们还是可以使用多线程进行提升效率。

# 同步与异步、阻塞与非阻塞

In [ ]:
# 同步，就是在发出一个功能调用时，在没有得到结果之前，该调用就不返回。也就是必须一件一件事做,等前一件做完了才能做下一件事。
# 例如普通B/S模式（同步）：提交请求->等待服务器处理->处理完毕返回 这个期间客户端浏览器不能干任何事
# 异步的概念和同步相对。当一个异步过程调用发出后，调用者不能立刻得到结果。
# 实际处理这个调用的部件在完成后，通过状态、通知和回调来通知调用者。
# 例如 ajax请求（异步）: 请求通过事件触发->服务器处理（这是浏览器仍然可以作其他事情）->处理完毕

# 1. 同步，就是我调用一个功能，该功能没有结束前，我死等结果。
# 2. 异步，就是我调用一个功能，不需要知道该功能结果，该功能有结果后通知我（回调通知）
# 3. 阻塞，就是调用我（函数），我（函数）没有接收完数据或者没有得到结果之前，我不会返回。
# 4. 非阻塞，就是调用我（函数），我（函数）立即返回，通过select通知调用者
# 同步IO和异步IO的区别就在于：数据拷贝的时候进程是否阻塞
# 阻塞IO和非阻塞IO的区别就在于：应用程序的调用是否立即返回

# 分布式、高并发、多线程

In [ ]:
# 分布式是从物理资源的角度去将不同的机器组成一个整体对外服务，技术范围非常管且难度非常大，有了这个基础，高并发、高吞吐等系统很容易构建；
# 高并发是从业务角度去描述系统的能力，实现高并发的手段可以采用分布式，也可以采用诸如缓存、CDN等，当然也包括多线程；
# 多线程则聚焦于如何使用编程语言将CPU调度能力最大化。
# 分布式与高并发系统，涉及到大量的概念和知识点，如果没有系统的学习，很容易会杂糅概念而辨识不清，在面试与实际工作中都会遇到困难。

# 多线程

In [ ]:
# 多线程编程有两种方式：
# 一种是通过Thread类对线程进行实例化，
# 另外一种是通过继承Thread类并重写其run方法。

In [9]:
import threading
import time

def do_something(content, sec):
    print("%s started" % content)
    time.sleep(sec)
    print("%s completed" % content)
    
def main():
    thread1 = threading.Thread(target=do_something, args=("First task", 2))
    thread2 = threading.Thread(target=do_something, args=("Second task", 4))
    start_time = time.time()
    thread1.start()
    thread2.start()
    print("Last time:%fs" % (time.time() - start_time))

main()
# 为什么这里的时间会是0s呢？
# 原因是因为当我们创建了两个线程并启动后，此时的程序共有三个线程，thread1和thread2为子线程，main函数的线程被称为主线程，
# 可在线程内部通过threading.current_thread()来获取当前线程信息，主线程会默认明名为'MainThread'，可在创建线程时使用参数name标记线程名。
# 当开始了两个子线程后，由于三个线程并行，主线程也要继续运行，而执行两个start()方法的时间很短，所以打印的时间是0s。
# 并且这里的输出结果包含了线程结束语句'... completed'，说明主线程运行结束，程序并没有退出，而是要等子线程运行结束后再退出。

First task startedSecond task started
Last time:0.046800s

First task completed
Second task completed


In [4]:
# 如何使得主线程退出后子线程自动退出呢？只需要对子线程调用setDaemon方法，将子线程标记成守护线程
import threading
import time

def do_something(content, sec):
    print("%s started" % content)
    time.sleep(sec)
    print("%s completed" % content)

def main():
    thread1 = threading.Thread(target=do_something, args=('First task', 2))
    thread2 = threading.Thread(target=do_something, args=('Second task', 4))
    thread1.setDaemon(True)  # 将子线程标记成守护线程
    thread2.setDaemon(True)  # 将子线程标记成守护线程
    start_time = time.time()
    thread1.start()
    thread2.start()
    print('Last time: %fs' % (time.time() - start_time))

main()
# ！！！在jupypterbook中无效，在pycharm中是正常的

# Thread对象的setDaemon(True)的作用：
# 1、如果主线程是永远都不会结束的，那设置一个线程为守护线程是没必要的，设不设置都一样。
# 2、什么时候需要设置为守护线程？如果希望子线程一直运行，可以把子线程的代码写在while True里面一直循环，
# 但同时要设置为守护线程，不然主线程结束了，子线程还一直运行，程序结束不了。

First task started
Second task startedLast time: 0.031200s

First task completed
Second task completed


In [2]:
# 如何将主线程阻塞，等子线程运行完成后再继续主线程，这样我们就可以获得两个线程并行运行的时间了。
import threading
import time

def do_something(content, sec):
    print("%s started" % content)
    time.sleep(sec)
    print("%s completed" % content)

def main():
    thread1 = threading.Thread(target=do_something, args=('First task', 2))
    thread2 = threading.Thread(target=do_something, args=('Second task', 4))
    start_time = time.time()
    thread1.start()
    thread2.start()
    thread1.join()  # 可以将主线程阻塞
    thread2.join()  # 可以将主线程阻塞
    print('Last time: %fs' % (time.time() - start_time))

main()
# 运行时间并不是串行的6s，而是两者中的大者，原因是当线程遇到IO操作或者time.sleep时，GIL会释放，将执行权留给其他线程。

First task started
Second task started
First task completed
Second task completed
Last time: 4.046227s


In [8]:
# 通过继承Thread类并重写其run方法进行多线程编程
import threading

class DoSomething(threading.Thread):
    def __init__(self, content, sec):
        super().__init__()
        self.content = content
        self.sec = sec
    def run(self):
        print('%s started' % self.content)
        time.sleep(self.sec)
        print('%s completed' % self.content)

def main():
    thread1 = DoSomething('First task', 2)
    thread2 = DoSomething('Second task', 4)
    thread1.start()  # 实际上就是运行其run方法
    thread2.start()
    thread1.join()
    thread2.join()

main()
# 方法适用于逻辑简单明确的情况，当代码逻辑复杂时，最好使用这种方法，方便代码维护。
# 可以看出，threading.Thread类的start方法，实际上就是运行其run方法。

First task started
Second task started
First task completed
Second task completed


# run()和start()

In [ ]:
# start():开始执行该线程。
# 对每一个线程对象来说它只能被调用一次，它安排对象在一个另外的单独线程中调用run()方法（而非当前所处线程）。
# 当该方法在同一个线程对象中被调用超过一次时，会引入RuntimeError(运行时错误)。
# run():定义线程功能的方法。（通常开发者应该在子类中重写）
# 标准run()方法调用了传递给对象的构造函数的可调对象作为目标参数，如果有这样的参数的话，顺序和关键字参数分别从args和kargs取得。

In [1]:
import threading
import time
 
def worker():
    count = 1
    while True:
        if count >= 4:
            break
        time.sleep(1)
        count += 1
        print("thread name = {}, thread id = {}".format(threading.current_thread().name,threading.current_thread().ident))
 
t1 = threading.Thread(target=worker,name="t1")
t2 = threading.Thread(target=worker,name='t2')
 
t1.start()
t2.start()
 
print("===end===")
# start()方法启动了两个新的子线程并交替运行，每个子进程ID也不同。

===end===
thread name = t1, thread id = 7368
thread name = t2, thread id = 6016
thread name = t2, thread id = 6016
thread name = t1, thread id = 7368
thread name = t2, thread id = 6016thread name = t1, thread id = 7368



In [2]:
import threading
import time
 
def worker():
    count = 1
    while True:
        if count >= 4:
            break
        time.sleep(1)
        count += 1
        print("thread name = {}, thread id = {}".format(threading.current_thread().name,threading.current_thread().ident))
 
t1 = threading.Thread(target=worker,name="t1")
t2 = threading.Thread(target=worker,name='t2')
 
t1.run()
t2.run()
 
print("===end===")
# 两个子线程都用run()方法启动，但却是先运行t1.run()，运行完之后才按顺序运行t2.run()，
# 两个线程都工作在主线程，没有启动新线程，因此，run()方法仅仅是普通函数调用

thread name = MainThread, thread id = 7136
thread name = MainThread, thread id = 7136
thread name = MainThread, thread id = 7136
thread name = MainThread, thread id = 7136
thread name = MainThread, thread id = 7136
thread name = MainThread, thread id = 7136
===end===


In [3]:
# start() 方法是启动一个子线程，线程名就是我们定义的name
# run() 方法并不启动一个新线程，就是在主线程中调用了一个普通函数而已。

# 锁

In [ ]:
# threading.Lock()
# 形成死锁的两种情况
# 死锁的两种解决方法

In [5]:
import threading

total = 0
lock = threading.Lock()
def add():
    global total
    global lock
    for i in range(1000000):
        lock.acquire()
        total += 1
        lock.release()

def desc():
    global total
    global lock
    for i in range(1000000):
        lock.acquire()
        total -= 1
        lock.release()

thread1 = threading.Thread(target=add)
thread2 = threading.Thread(target=desc)
thread1.start()
thread2.start()
thread1.join()
thread2.join()

print(total)
# threading.Lock()实例化了一个锁对象，锁对象有两个方法：acquire和release，分别是获得锁和释放锁。
# 当一个线程获得所时，另外一个线程在acquire处阻塞，直到当前锁执行release被释放后才可以和其他线程共同争夺锁。
# acquire和release之间的代码段执行时不会切换到其他线程，保证了操作的完整性。

0


In [6]:
# 用锁也存在问题，首先就是性能问题，在上面的例子中，不使用锁运行的执行时间是0.15秒，而使用锁执行时间是2.35秒，足足慢了15倍。
# 另外一个问题被称为死锁。当一个线程调用子程序时，如果这个子程序也需要加锁，则会出现这个问题：

In [ ]:
# 下列程序会形成死锁
import threading
import time

lock = threading.Lock()

def do_something():
    global lock
    lock.acquire()
    do_sub_task()
    lock.release()

def do_sub_task():
    global lock
    lock.acquire()
    time.sleep(2)
    lock.release()

thread = threading.Thread(target=do_something)
thread.start()
thread.join()
# 程序会在do_sub_task的首句阻塞，因为该函数试图去获取锁，但是锁并没有释放

In [ ]:
# 一种死锁情况称为互相等待，下列程序会形成死锁：
import threading
import time

lock1 = threading.Lock()
lock2 = threading.Lock()

def do_something1():
    lock1.acquire()
    time.sleep(2)
    lock2.acquire()
    print('Something 1 started')
    time.sleep(2)
    lock1.release()
    lock2.release()
    print('Something 1 ended')

def do_something2():
    lock2.acquire()
    time.sleep(3)
    lock1.acquire()
    print('Something 2 started')
    time.sleep(3)
    lock2.release()
    lock1.release()
    print('Something 2 ended')

thread1 = threading.Thread(target=do_something1)
thread2 = threading.Thread(target=do_something2)
thread1.start()
thread2.start()
thread1.join()
thread2.join()
# 原因在于两个线程获得两个锁的顺序是相反的，当do_something1运行时获得lock1，然后执行time.sleep(2)使得GIL释放去执行do_something2。
# do_something2获得lock2后，同样执行time.sleep(3)使得GIL释放去执行do_something1，do_something1此时需要获得lock2才能继续执行，
# 然而lock2在do_something2处，未释放无法获得。同理do_something2需要获得的lock1在do_something1处，也无法获得。
# 所以就出现了两个线程互相等待的情况。如果将其中某个线程获得的锁的顺序交换，程序就能正常执行。

In [ ]:
# 解决死锁的方法有两种：
# 一种是通过threading.Lock()再实例化一把锁，使得do_something和do_sub_task所需要的锁不是同一把，
# 这样即使do_something获取了锁，do_sub_task也能够获得另外的锁。但是这种方式的问题是当这种情况出现很多，锁就很难管理。
# 另外一种是使用threading.RLock，这种锁可以重复获得，只要释放的次数等于获得的次数即可。将上面代码中的Lock换成RLock即可。

# 生产者消费者问题、queue

In [ ]:
# queue

In [1]:
# 生产者消费者问题是多线程中一个很经典并发协作的问题，
# 这个问题主要包含两类线程，一个是生产者用于生产数据，另一个是消费者用于消费数据，
# 两者操作同一个数据共享区域，这种模型在编程中非常常见，比如爬虫，生产者负责爬取链接，消费者负责解析链接所指向的网页内容。
# 这种模型需要满足下面的两个特征：
# 消费者在数据共享区域为空时阻塞，直到共享区域出现新数据。
# 生产者在数据共享区域满时阻塞，直到数据共享区出现空位。

In [ ]:
import threading
import time
import random
MAX_BUFF_LEN = 5

buff = []
lock = threading.Lock()

class Producer(threading.Thread):
    def run(self):
        global buff
        while True:
            lock.acquire()
            if len(buff) < MAX_BUFF_LEN:
                # 如果共享区域未满，生产数据
                num = random.uniform(0, 5)
                buff.append(num)
                print('生产者向共享区域加入%f' % num)
                lock.release()
            time.sleep(random.uniform(0, 10))  # uniform() 方法将随机生成下一个实数，它在 [x, y] 范围内。

class Consumer(threading.Thread):
    def run(self):
        global buff
        while True:
            lock.acquire()
            if buff:
                # 如果共享区非空，消费数据
                num = buff.pop(0)
                print('消费者消费掉%f' % num)
                lock.release()
            time.sleep(random.uniform(0, 10))

producer = Producer()
consumer = Consumer()
producer.setDaemon(True)
consumer.setDaemon(True)
try:
    producer.start()
    consumer.start()
    producer.join()
    consumer.join()
except KeyboardInterrupt:
    print('程序强制结束！')
# 在程序中设置两个进程为守护进程，并捕捉KeyboardInterrupt错误，一旦捕捉到就结束主线程，同时结束两个子线程。
# 上面是一个生产者消费者模型的一个简单实现，通过共享变量的方式使两个线程互相通信来达成一致。
# 共享变量是线程间通信的常用方法，只要记得在对共享变量进行操作时加锁，程序就不会有问题。

生产者向共享区域加入4.617116
消费者消费掉4.617116


In [1]:
# 上面的代码也有问题，在于这种代码通过无限对共享变量访问的方式进行判断空还是满，这样也降低了效率。
# 因为其中一个程序在明明知道buff满了或者空了的情况下还要进行无意义的循环，由于GIL机制，它会和其他线程争夺执行权。
# 如果某一方在判断buff满了或者空了的情况下主动阻塞，直到另外一方通知它，它才恢复，这样就能最大化的效率。

In [ ]:
# Python中threading中的Condition类就是来帮助我们完成这件事的。它的wait和notify方法能够阻塞和通知一个线程
import threading
import time
import random
MAX_BUFF_LEN = 5

buff = []
condition = threading.Condition()

class Producer(threading.Thread):
    def run(self):
        global buff
        while True:
            condition.acquire()
            if len(buff) < MAX_BUFF_LEN:
                # 如果共享区域未满，生产数据
                num = random.uniform(0, 5)
                buff.append(num)
                print('生产者向共享区域加入%f' % num)
                condition.notify()
            else:
                # 如果共享区满，停止生产
                print('共享区满，生产者阻塞！')
                condition.wait()
            condition.release()
            time.sleep(random.uniform(0, 10))

class Consumer(threading.Thread):
    def run(self):
        global buff
        while True:
            condition.acquire()
            if buff:
                # 如果共享区非空，消费数据
                num = buff.pop(0)
                print('消费者消费掉%f' %num)
                condition.notify()
            else:
                # 如果共享去空，停止消费
                print('共享区空，消费者阻塞！')
                condition.wait()
            condition.release()
            time.sleep(random.uniform(0, 10))

producer = Producer()
consumer = Consumer()
producer.setDaemon(True)
consumer.setDaemon(True)
try:
    producer.start()
    consumer.start()
    producer.join()
    consumer.join()
except KeyboardInterrupt:
    print('程序强制结束！')
# acquire方法实际上是获得锁，wait方法将线程阻塞，实际上是将锁释放。
# 当一个线程调用notify方法时，另一个线程就被唤醒，但是这时候这个线程并没有调用wait或者release方法释放锁，
# 因此另一个线程虽然醒过来了但是还是没有执行，直到这个线程将锁释放。

In [ ]:
# 在使用共享变量的时候，需要时刻注意是否线程安全，非常不方便。
# 好在是Python中提供了一个Queue类，它是线程安全的，有了它我们可以把注意力放在如何实现代码逻辑上，而不是过多的注意到线程安全上。
# 在Python2.7中该模块名为Queue，而在Python3.6中该模块名为queue。使用Queue类改进的代码如下：
import threading
import time
import random
from queue import Queue

MAX_BUFF_LEN = 5

buff = Queue(MAX_BUFF_LEN)
condition = threading.Condition()

class Producer(threading.Thread):
    def run(self):
        global buff
        while True:
            num = random.uniform(0, 5)
            buff.put(num)
            print('生产者向共享区域加入%f' % num)
            time.sleep(random.uniform(0, 10))

class Consumer(threading.Thread):
    def run(self):
        global buff
        while True:
            num = buff.get()
            print('消费者消费掉%f' %num)
            time.sleep(random.uniform(0, 10))

producer = Producer()
consumer = Consumer()
producer.setDaemon(True)
consumer.setDaemon(True)
try:
    producer.start()
    consumer.start()
    producer.join()
    consumer.join()
except KeyboardInterrupt:
    print('程序强制结束！')
# Queue是一个FIFO队列，它的get方法和put方法分别是入队和出队，在入队和出队时获取了锁以保证线程安全，
# 如果队列空或者满，默认情况下get方法和put方法自动阻塞。阻塞和唤醒的方式实质上是调用了Condition类的wait和notify方法

# 线程池

In [2]:
# 使用线程池管理线程能够使主线程可以获得某一线程的状态以及返回值，当一个线程完成的时候主线程就能立知道。
# 这里我们使用的线程池类是ThreadPoolExecutor，它在concurrent.futures下。
# concurrent.futures中还包括了ProcessPoolExecutor进程池对象，这个包的设计让多线程和多进程的接口一致。

In [5]:
from concurrent.futures import ThreadPoolExecutor
import time

def do_something(name, sec):
    print('Start doing %s' % name)
    time.sleep(sec)
    print('%s completed' % name)
    return name

executor = ThreadPoolExecutor(max_workers=2)  # 最大同时运行的线程个数
task = executor.submit(do_something, 'A', 2)
print(task.done())  # 该方法非阻塞
print(task.done())  # 该方法非阻塞
print(task.result())  # 该方法阻塞
print(task.done())  # 该方法非阻塞
# 首先需要实例化一个线程池对象，ThreadPoolExecutor类包含一个参数max_workers，表示最大同时运行的线程个数。
# 线程池中可以创建任意多个线程，但是同时能运行的个数为max_workers，其他线程需要等当前正在运行的max_workers个线程运行完成才能运行。
# 线程池对象的submit方法可传入一个函数句柄及它的参数，参数依次排列。一旦调用submit方法，线程就已经开始执行，并返回一个Future对象。
# 可调用Future对象的done方法查看线程是否执行完成，该方法非阻塞。还可以调用result方法获得线程的返回值，该方法阻塞直到线程结束得到返回值。

Start doing AFalse

False
A completed
A
True


In [7]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import random
import time


def do_something(name, sec):
    print('Start doing %s' % name)
    time.sleep(sec)
    print('%s completed' % name)
    return name


executor = ThreadPoolExecutor(max_workers=2)  # 最大同时运行的线程个数
all_task = [executor.submit(do_something, 'task_%d' % i, random.uniform(2, 6)) for i in range(5)]
# wait(fs=all_task, return_when=ALL_COMPLETED)  # concurrent.futures还提供了wait方法，用于阻塞主线程
for future in as_completed(all_task):
    data = future.result()
    print(data)
# as_completed是一个生成器，它会生成已经完成的线程的future对象。
# 先执行完成的线程的future对象会先被生成，直到所有线程结束，最后一个线程的future对象被生成。
# 从结果来看，由于每次的线程切换不同，执行结果也不同。

Start doing task_0
Start doing task_1
task_0 completed
Start doing task_2
task_0
task_1 completed
Start doing task_3task_1

task_2 completed
Start doing task_4task_2

task_3 completed
task_3
task_4 completed
task_4


In [8]:
# for data in executor.map(do_something, ['task_%d' % i for i in range(10)], [random.uniform(2,6) for i in range(10)]):
#     print(data)
# 和之前的as_completed方法不同，map生成器是按照参数的顺序返回的，但是线程执行依然是无序的。
# 而且map返回的是线程的返回值，不是Future对象。在实践中最常用的还是第一种方法。

In [ ]:
# concurrent.futures还提供了wait方法，用于阻塞主线程。其用法是：
# from concurrent.futures import wait, ALL_COMPLETED, FIRST_COMPLETED, FIRST_EXCEPTION
# wait(fs=all_task, return_when=ALL_COMPLETED)
# 第一参数fs是需要等待的线程列表，还有一个可选参数是return_when，即停止阻塞的条件，默认是ALL_COMPLETED，即所有线程完成。
# 除此之外还包括：FIRST_COMPLETED（第一个线程执行完成后）、FIRST_EXCEPTION（在子线程中第一次出现抛出错误后）